In [10]:
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

import concurrent.futures

# getting data

things we did to clean data:

1. shamelessly copy:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

2. we removed the tickers with $

3. we added exception handeling that tries similer strings to the input (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)
4.  we just drop exceptions that are found when getting the data from online  (these are diffrent every day)

# how are we keeping consistency? 
since the avilble stocks may change depending on the source of the data some extra steps need to be done in order to
alow saves to work seemlesly 


basic plan:

1. clean the data as much as possible 

2. save the embedings weights with their ticker string

3. backup everything on git to alow us to fix it



In [53]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

In [8]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
sav_set.discard("") 

ticker_list=list(sav_set)

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1583 unqualified stock symbols...
There are 10615 qualified stock symbols...


In [9]:
start='01/01/2020' 
end='09/14/2022'

In [12]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()

def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [13]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
100
error number 1: name=ZAZZT
error number 2: name=EMCH
200
300
400
500
error number 3: name=ZTEST
600
700
error number 4: name=CTEST.G
800
error number 5: name=ZVZZT
900
1000
1100
error number 6: name=IBO
1200
1300
1400
error number 7: name=EMP
error number 8: name=TAFI
1500
error number 9: name=EMGC
1600
error number 10: name=CTEST
1700
1800
1900
2000
2100
2200
error number 11: name=KCAC.U
2300
2400
error number 12: name=ZXZZT
2500
error number 13: name=CTEST.O
2600
2700
2800
error number 14: name=BLLD
error number 15: name=ZBZX
2900
3000
3100
error number 16: name=EAI
error number 17: name=YEAR
3200
error number 18: name=CBX
error number 19: name=ZVZZC
3300
error number 20: name=ATEST.C
3400
3500
3600
error number 21: name=ATEST.A
3700
error number 22: name=CTEST.E
3800
error number 23: name=ZXYZ.A
3900
error number 24: name=CBO
4000
error number 25: name=ZCZZT
4100
error number 26: name=ZEXIT
4200
4300
4400
error number 27: name=KCA.U
4500
error number 28: 

In [15]:
data_df=pd.DataFrame(data).transpose()
data_df

,KNX,ISRA,SAH,SINT,DMYS,ESI,NBCT,ORIA,EGAN,BOC,...,GLHAU,HWBK,QARP,CTAQU,IPGP,RYE,OVLY,GOLF,BLU,GMRE
2020-01-02,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,NaN,1.000000,1.000000,...,NaN,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2020-01-03,0.989997,0.988431,0.976478,1.034247,NaN,0.991453,NaN,NaN,0.988861,1.005145,...,NaN,0.983574,0.990445,NaN,0.968187,1.002291,0.993782,1.010836,1.063226,1.018504
2020-01-06,0.991349,0.987867,0.943738,1.041096,NaN,0.988889,NaN,NaN,0.974010,1.010290,...,NaN,0.981619,0.993410,NaN,0.978086,1.012497,0.991710,1.021053,1.070968,1.033924
2020-01-07,1.008921,0.994357,0.948188,1.150685,NaN,0.999145,NaN,NaN,0.960396,1.001871,...,NaN,0.978882,0.992092,NaN,0.983752,1.021245,0.970466,0.991641,1.027097,1.028527
2020-01-08,1.002703,0.997178,0.943738,1.239726,NaN,1.002564,NaN,NaN,0.972772,1.011693,...,NaN,0.980837,0.996376,NaN,0.983411,0.999792,0.974093,0.990093,1.032258,1.043947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,1.379588,1.157333,1.613864,0.276027,0.984016,1.646224,0.984201,1.023958,1.092822,1.228251,...,1.021630,1.026023,1.256502,0.995,0.610117,1.450781,0.984163,1.570633,1.504516,0.953409
2022-09-08,1.375992,1.163347,1.573082,0.280822,0.985015,1.674304,0.987963,1.022917,1.092822,1.210477,...,1.047485,1.012500,1.265236,0.993,0.612916,1.456849,0.976646,1.585344,1.516129,0.954295
2022-09-09,1.407799,1.188550,1.558939,0.284247,0.983017,1.696242,0.998258,1.025000,0.936881,1.235267,...,1.047485,0.985033,1.287355,0.993,0.629301,1.495638,0.976646,1.609331,1.501936,0.967586
2022-09-12,1.408352,1.206307,1.622744,0.282192,0.983017,1.717303,1.005227,1.022917,0.991337,1.246024,...,1.047485,0.993062,1.296527,0.993,0.634558,1.529659,0.971814,1.609970,1.477419,0.988852


In [55]:
ticker_list=data_df.columns
print(f"{len(symbols)-len(ticker_list)} stocks were droped")

1634 stocks were droped


# preprocessing 


1. ticker: this is also used to select which stocks come to the model

2. price: we take the ratio: (importantly we did drop some potentialy important indicators)

3. date: the day and month are floating point numbers. while year and day of the week are embedings.



In [57]:
look=layers.StringLookup(vocabulary=ticker_list)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True,num_oov_indices=0)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)
day_embed=layers.Embedding(7,10)

In [87]:
dataset=tf.data.Dataset.from_tensor_slices(ticker_list)
exmple=dataset.batch(17).take(1).get_single_element()
exmple

<tf.Tensor: shape=(17,), dtype=string, numpy=
array([b'KNX', b'ISRA', b'SAH', b'SINT', b'DMYS', b'ESI', b'NBCT',
       b'ORIA', b'EGAN', b'BOC', b'APRZ', b'MMAT', b'LDP', b'MKSI',
       b'NLIT', b'GCT', b'CNTQU'], dtype=object)>

In [111]:
def choose_stocks(x):
    a=tick_embed(look(ticker_list))
    b=tick_embed(look(x))

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,3)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    return [[data_df[x] for x in sub] for sub in tickers]

%timeit a=choose_stocks(exmple)

pd.DataFrame(a[0])

31.6 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


,2020-01-02,2020-01-03,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,2020-01-13,2020-01-14,2020-01-15,...,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-06,2022-09-07,2022-09-08,2022-09-09,2022-09-12,2022-09-13
KNX,1.0,0.989997,0.991349,1.008921,1.002703,1.000811,1.007029,1.012436,1.027034,1.026494,...,1.420458,1.393694,1.408076,1.391204,1.355249,1.379588,1.375992,1.407799,1.408352,1.375716
AWP,1.0,1.000000,1.004630,1.003086,1.003086,1.004630,1.012346,1.026235,1.041667,1.041667,...,0.919759,0.917893,0.919759,0.902968,0.904834,0.917893,0.916027,0.919759,0.925356,0.908565
BCML,1.0,0.986726,0.980531,0.973451,0.973894,0.976991,0.980531,0.978319,0.983628,0.985398,...,0.828108,0.827219,0.816551,0.831220,0.819218,0.825441,0.837443,0.844555,0.848111,0.828997


In [142]:
def thread_distribute(l,f,n=1):
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [131]:
def time_encode(a):
    return (a.day_of_week,a.day/30,a.month/12,(a.year-2020))
    
time_encode(data_df.index[0])

(3, 0.06666666666666667, 0.08333333333333333, 0)

In [138]:
x=thread_distribute(a,time_encode,2)
x=[thread_distribute(x,lambda x: x[i],2) for i in range(4)]
x=[tf.constant(x) for x in x]
[x.shape for x in x]

[TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680]),
 TensorShape([17, 680])]

In [143]:
[x.dtype for x in x]

[tf.int32, tf.float32, tf.float32, tf.int32]